In [1]:
#색인하기
import re
from os import listdir

path = 'naver'

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return rst

def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))

In [2]:
# indexer 하기
#regex 사용하기
from nltk.tokenize import regexp_tokenize
from collections import Counter

tokens = list()

for file in fileids('naver'):
    with open(file, 'r', encoding='utf8') as fp:
        tokens.extend(regexp_tokenize(fp.read(), r'\b\w+\b'))

tokens = Counter(tokens)

In [5]:
len(tokens), sum(tokens.values())

(44872, 131710)

In [11]:
#controlled voca
CV = list(tokens.keys())

In [20]:
D = fileids('naver')

In [21]:
len(CV), D
#이미 과하게 크다. 안돌아갈수 있다

(44872,
 ['naver/0000015228.txt',
  'naver/0000015343.txt',
  'naver/0000015369.txt',
  'naver/0000015398.txt',
  'naver/0000020088.txt',
  'naver/0000020663.txt',
  'naver/0000021218.txt',
  'naver/0000022648.txt',
  'naver/0000046326.txt',
  'naver/0000047901.txt',
  'naver/0000049955.txt',
  'naver/0000050638.txt',
  'naver/0000054197.txt',
  'naver/0000120141.txt',
  'naver/0000353463.txt',
  'naver/0000361949.txt',
  'naver/0000361955.txt',
  'naver/0000362047.txt',
  'naver/0000362050.txt',
  'naver/0000362063.txt',
  'naver/0000362078.txt',
  'naver/0000430333.txt',
  'naver/0000430354.txt',
  'naver/0000430365.txt',
  'naver/0000430389.txt',
  'naver/0000430464.txt',
  'naver/0000430497.txt',
  'naver/0000430508.txt',
  'naver/0000430514.txt',
  'naver/0000430536.txt',
  'naver/0000621113.txt',
  'naver/0000621466.txt',
  'naver/0000622324.txt',
  'naver/0000622489.txt',
  'naver/0000622492.txt',
  'naver/0000622502.txt',
  'naver/0000622594.txt',
  'naver/0000622635.txt',
  'n

In [22]:
#그래서 list 를 하나 만들어보겠다.
#Document Term Math
# 전체 문서개수만큼 D가 있고 전체 단어개수만큼 t가 있고, 그만큼을 0이라고 하자. 
DTM = [[0 for t in range(len(CV))]for d in range(D)]

TypeError: 'list' object cannot be interpreted as an integer

In [23]:
len(DTM), len(DTM[0])

(387, 44872)

In [24]:
#D를 돌려보자
for i, file in enumerate(D):
    with open(file, 'r', encoding='utf8') as fp:
        #boolean 처리
        for t in regexp_tokenize(fp.read(), r'\b\w+\b'):
            #index 찾기
            if t in CV:
                j = CV.index(t)
                DTM[i][j] = 1

In [27]:
#문서별로 몇개의 단어가 사용됐냐
#cv는 0번째 단어를 의미 -> sum 할거임
# list(map(lambda d:sum(d)/len(d), [d[0] for d in DTM]))

In [30]:
sum([d[3]/len(D) for d in DTM])

0.002583979328165375

In [35]:
#불리언 검색
q = '대선 후보 인터뷰' #쿼리 <- 작은문서
rst = list()
#작은 문서 풀어보기
for wi in regexp_tokenize(q, r'\b\w+\b'):
    #DTM을 돌면서 뒤에 있는 term 을 본다
    #쿼리에 포함되어 있는 단어들 wi 만큼. CV에 있는지를 내부적으로 검사를 한다.
    #DTM = (387, 44872)
    #이 개수가 많아질수록 많이 돌아본다.
    #문서 전체 도는 구간(DTM의 행이 = > 문서 셋)
    for i, d in enumerate(DTM):
        #단어 전체를 도는 구간. (각 행이 => 단어 셋)
        for j, wj in enumerate(d):
            if CV[j]== wi:
                rst.append(i)
                break

#근데 이거는 아주 안좋은 구조다. 
#이걸 가지고 Time Complexity를 잰다고 하면, 쿼리에서 뽑은 개수 곱하기 전체 문서에서 개수 곱하기 문서니까... 
# TimeComplexity = O(|Q|*D*|D)
# 그니까 저만큼 공간이 필요한거다. 



In [36]:
len(list(set(rst))), len(D)
#표현이 다 들어간걸 알 수가 잇다. 

(387, 387)

In [37]:
# 쿼리마다 리스트를 따로 만들면된다고 한다.
q = '대선 후보 인터뷰' #쿼리 <- 작은문서
rst = list()

#작은 문서 풀어보기
for wi in regexp_tokenize(q, r'\b\w+\b'):
    #DTM을 돌면서 뒤에 있는 term 을 본다
    #쿼리에 포함되어 있는 단어들 wi 만큼. CV에 있는지를 내부적으로 검사를 한다.
    #DTM = (387, 44872)
    #이 개수가 많아질수록 많이 돌아본다.
    #문서 전체 도는 구간(DTM의 행이 = > 문서 셋)
    candi = list()
    for i, d in enumerate(DTM):
        #단어 전체를 도는 구간. (각 행이 => 단어 셋)
        for j, wj in enumerate(d):
            if CV[j]== wi:
                candi.append(i)
                break

    rst.append(candi)

In [ ]:
len(list(set(rst[0]))).intersection(rst[i])).intersection()

## solution : inverted index
DTM을 TDM 으로 바꾸자는 거임. 트랜스포즈로 바꾸자는거임. 
근데 메모리 안올라갈거다. 

In [ ]:
TDM = [[0 for d in range()]]

In [ ]:
for i, file in enumerate(D):
    with open(file, 'r', encoding='utf8') as fp:

        for t in regexp_tokenize(fp.read(), r'\b\w+\b'):
            #index 찾기
            if t in CV:
                j = CV.index(t)
                DTM[i][j] = 1

In [ ]:
q = '대선 후보 인터뷰' 

rst = list()
for wi in regexp_tokenize(q, r'\b\w+\b'):
    # 아까와 달리 Timecomplexity : Q(|Q|*|D=1|)
    # Timecomplexity : Q(|Q|*avg|{d ㅌ D, if d=1})
    j = CV.index(wi)
    # TDM[j] #이게 검색후보
    rst.extend(list(filter(lambda x:x==1, TDM[j])))

In [ ]:
sum(map(lambda x:sum(list(filter(lambda y:y==1, x))), TDM)) / len(TDM)
#구조를 이렇게 바꾸니 시간 복잡도가 많이 줄어든게 느껴진다. 

In [ ]:
len(set(rst))

In [ ]:
CV.index(wi), sum(TDM[7])

In [ ]:
#이번에는 for 문이 한번 돈다. 

# 내일은 이 코드들을 더 어렵게, low 레벨로 점점 코드를 바꿔볼거임